#  Proyecto Final  

Pablo Izquierdo Conde

Jorge Pose Eiroa

La primera parte del Proyecto consiste en obtener las páginas con las que vamos a trabajar. Para ello seleccionamos dos categorías de la Wikipedia, que en nuestro caso, y por ser las categorías iniciales muy disjuntas, el profesor nos ha permitido escoger dos subcategorías de la categoría "Sports".

In [7]:
import wikipediaapi as wk
import numpy as np
import copy

wiki = wk.Wikipedia('en')

# Cogemos las categorías seleccionadas
cat0 = wiki.page("Category:Esports")
cat1 = wiki.page("Category:Combat_sports")

Una vez seleccionadas las categorías es necesario obtener las páginas que cuelgan de ellas, así como algunas de las páginas que cuelgan de las subcategorías de las categorías elegidas. Esto es lo que hace la siguiente función, a la que le hemos puesto una profundidad de 3 niveles, que es más que suficiente para obtener el número de páginas necesarias.

In [8]:
# Función que busca recursivamente en las categorías y nos devuelve una lista con las páginas
def print_categorymembers(categorymembers, level=0, max_level=2, pages = []):
    for c in categorymembers.values():
        if c.ns == 0: # Sacamos las páginas (ns=0)
            pages.append(c)
        if c.ns == wk.Namespace.CATEGORY and level <= max_level:
            print_categorymembers(c.categorymembers, level + 1,1, pages)
p0 = [] #len=1227
p1 = []#len=4288
print_categorymembers(cat0.categorymembers,0,1, p0)
print_categorymembers(cat1.categorymembers,0,1, p1)

Una vez obtenidas las páginas, procedemos a obtener el texto que usaremos como corpus para el resto del proyecto, tomando 500 páginas aleatorias de cada categoría. Este proceso va a llevar un rato, ya que es necesario recorrer todas las páginas y descargar el texto.

In [12]:
indices0 = np.random.permutation(len(p0))[:500]
indices1 = np.random.permutation(len(p1))[:500]

# Sacamos el texto de las listas de páginas
corpus0 = []
corpus1 = []

# Corpus 0 train
print('Corpus 0')
for n,i in enumerate(indices0):
    if not n%100:
        print('\rPage', n, 'out of', len(indices0), end='', flush=True)
    corpus0.append(p0[i].text)
     
# Corpus 1 train    
print('\nCorpus 1')
for n,i in enumerate(indices1):
    if not n%100:
        print('\rPage', n, 'out of', len(indices1), end='', flush=True)
    corpus1.append(p1[i].text)

corpustotal=copy.deepcopy(corpus0)
corpustotal.extend(corpus1)


Corpus 0
Page 400 out of 500
Corpus 1
Page 400 out of 500

# 2. Procesado de texto

    1. Tokenization
    2. Homogeneization
    3. Cleaning
    4. Vectorization

In [13]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim

wnl = WordNetLemmatizer()
stopwords_en = stopwords.words('english')
palabros = ['wa','also','ha']
stopwords_en.extend(palabros)

In [15]:
def getCorpusClean(corpus):
    corpus_clean=[]
    for text in corpus:
        tokens = word_tokenize(text)
        tokens_filtered = [el.lower() for el in tokens if el.isalnum() and not el.isdigit()] # Nos quedamos solo con palabras, quitamos numeros
        tk_lemmat = [wnl.lemmatize(el) for el in tokens_filtered]
        tk_clean = [tk for tk in tk_lemmat if tk not in stopwords_en]
        corpus_clean.append(tk_clean)
    return corpus_clean

corpus_clean = getCorpusClean(corpustotal)

In [16]:
def getCorpusBow(corpus_clean, no_below = 5, no_above = .75):
    D = gensim.corpora.Dictionary(corpus_clean)
    D.filter_extremes(no_below=no_below, no_above=no_above, keep_n=25000)
    corpus_bow = [D.doc2bow(doc) for doc in corpus_clean]
    return corpus_bow, D
    
corpus_bow, D = getCorpusBow(corpus_clean)

# 3. Modelado de tópicos con LDA

El modelado de tópicos se realiza teniendo en cuenta los documentos de ambas categorías, para generar así tópicos que permitan distinguir documentos pertenecientes a una y a otra.

In [18]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

def getLDAModel(corpus_bow, D, num_topics):
    ldag = gensim.models.ldamodel.LdaModel(corpus=corpus_bow, id2word=D, num_topics = num_topics, passes=20)
    return ldag

num_topics = 50
ldag = getLDAModel(corpus_bow, D, num_topics)

Para visualizar de una manera más clara los tópicos se traza el siguiente gráfico. En este se observa como los tópicos que sobresalen en la misma categoría se encuentran más próximos, y la influencia de cada término en los distintos tópicos (y por ende en las categorías).

In [19]:
def visTopics(ldag, corpus_bow, D):    
    vis_data = gensimvis.prepare(ldag, corpus_bow, D)
    return pyLDAvis.display(vis_data)   

visTopics(ldag, corpus_bow, D)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


Una vez hallados los tópicos, es necesario pasarlos a una matriz para poder ser utilizados por el clasificador.

In [21]:
def getExpandedMatrix(corpus_bow, ldag, num_topics):
        reduced_corpus = [el for el in ldag[corpus_bow[:]]]
        X = gensim.matutils.corpus2dense(reduced_corpus, num_topics).T
        return X


Xtotal = getExpandedMatrix(corpus_bow, ldag, num_topics)

y0 = np.zeros((500,1)) #antes 1000
y1 = np.ones((500,1)) #antes 1000
Stotal = np.vstack((y0,y1))

# 4. Implementación del clasificador

In [28]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import sklearn.model_selection as modselect
from sklearn.metrics import mean_squared_error, accuracy_score

X_tr, X_test, S_tr, S_test = modselect.train_test_split(Xtotal, Stotal, test_size=0.20, random_state=42)

In [36]:
svcparams = {'C': [100, 200, 350, 500, 1000, 1200, 1500 ],
     'gamma': [0.3, 0.5, 0.7, 0.9, 1.1]}

svcGrid = modselect.GridSearchCV(clf, svcparams, cv=20, verbose=1, scoring='accuracy', n_jobs=-2).fit(Xtotal,np.ravel(Stotal))

Fitting 20 folds for each of 35 candidates, totalling 700 fits


[Parallel(n_jobs=-2)]: Done 146 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done 700 out of 700 | elapsed:    4.5s finished


In [37]:
print('best accuracy score', svcGrid.best_score_)
print('best_params', svcGrid.best_params_)

best accuracy score 0.922
best_params {'C': 100, 'gamma': 0.3}


In [38]:
clf = svm.SVC(kernel = 'rbf', C=100, gamma=0.3)
clf.fit(X_tr,np.ravel(S_tr))

print('Accuracy: ',accuracy_score(S_test, clf.predict(X_test)))

Accuracy:  0.905
